In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import odeint
from matplotlib import pyplot as plt

In [2]:
Vrna_data = np.array([0.113636, 0.000000, 0.090909, 0.204545, 0.204545, 0.204545, 0.204545, 0.500000, 0.704545, 
                      0.409091, 0.500000, 0.500000, 0.613636, 0.500000, 0.295455, 0.795455, 0.704545, 0.409091, 
                      0.318182, 0.295455, 0.409091])
Vrna_unvacc_data = np.array([0.000000, 0.295455, 0.795455, 1.000000, 1.681818, 2.090909, 1.500000, 2.295455, 
                    2.204545, 2.477273, 2.704545, 2.590909, 2.477273, 2.386364, 2.090909, 1.704545, 1.613636, 
                    1.681818, 2.000000, 1.590909, 1.295455])
Vinf_data = np.array([0.011904762, 0.011904762, 0.011904762, 0.011904762, 0.107142857, 0.011904762, 0.011904762, 
             0.011904762, 0.107142857, 0.107142857, 0.107142857, 0.107142857, 0.30952381, 0.107142857, 
             0.011904762, 0.011904762, 0.011904762, 0.011904762, 0.107142857, 0.011904762, 0.011904762])
Vinf_unvacc_data = np.array([0.011904762, 0.011904762, 0.30952381, 0.30952381, 0.5, 0.702380952, 0.511904762, 
                             0.797619048, 0.702380952, 0.80952381, 0.80952381, 0.702380952, 0.511904762, 
                             0.214285714, 0.202380952, 0.214285714, 0.107142857, 0.011904762, 0.107142857, 
                             0.107142857, 0.107142857])
Ab_data = np.array([493.3333, 426.6666667, 443.3333333, 520, 443.3333333]) #update data
Ab_unvacc_data = np.array([273.3333333, 326.6666667, 433.3333333, 750, 476.6666667])
t1 = [0, 14, 26, 28] 
t2 = [28, 30, 30.5, 31, 31.5, 32, 32.5, 33, 33.5, 34, 34.5, 35, 35.5, 36, 36.5, 37, 37.5, 38, 38.5, 
                        39, 39.5, 40, 40.5, 41, 41.5, 56, 183]
crna = 0.0068
cinf = 0.2

In [3]:
#vaccinated IC's:
Va0 = 0 #/mL #should be 0
Vinf28 = 0 #pfu/mL
Vrna28 = 0 #RNA/mL
A0 = 363 
M0 = 0
I0 = 0
T0 = 1
vaccinated_initial_condition = [T0, I0, Va0, Vinf28, Vrna28, M0, A0]

In [4]:
#unvaccinated IC's:
Va_un0 = 0
Vinf_28 = 10**(4.5) #pfu/mL
Vrna_28 = 10**(4.5) #RNA/mL
Aun0 = 250 #ish
Mun0 = 0
Iun0 = 0
Tun0 = 0
unvaccinated_initial_condition = [Tun0, Iun0, Va_un0, Vinf_28, Vrna_28, Mun0, Aun0]

In [5]:
#IGs:
beta = 10**(-3)
Delta = 2
alpha = 0.5
p = 10**3
#cinf = 0.5
K = 0.1
v = 10
#crna = 0.005
gamma = 0.1
rho = 0.1
N = 800
lambdaa = 1
mu = 0.005
initial_guess = [10**(-4), 2, 0.5, 10**4, 0.1, 10, 0.1, 0.1, 800, 0.1, 0.005]
#change these up

In [6]:
def model(variables, t, beta, Delta, alpha, p, K, v, gamma, rho, N, lambdaa, mu):
    T = variables[0]
    I = variables[1]
    Va = variables[2]
    Vinf = variables[3]
    Vrna = variables[4]
    M = variables[5]
    A = variables[6]
    dTdt = -beta * T * Vinf #actual variables 
    dIdt = beta * T * Vinf - Delta * I #make sure these are right
    dVadt = -alpha * Va
    dVinfdt = p * I - cinf * Vinf - K * A * Vinf
    dVrnadt = v * p * I - crna * Vrna - K * A * Vrna
    dMdt = (gamma * (Vrna + Va) + rho * M) * (1 - M / N)
    dAdt = lambdaa * (Vrna + Va) * M * (1 - A / N) - mu * A
    return [dTdt, dIdt, dVadt, dVinfdt, dVrnadt, dMdt, dAdt]

In [7]:
def ssr_vacc(params): #list parameters
    beta = params[0]
    #print(type(beta))
    Delta = params[1]
    alpha = params[2]
    p = params[3]
    #cinf = params[4]
    K = params[4]
    v = params[5]
    #crna = params[7]
    gamma = params[6]
    rho = params[7]
    N = params[8]
    lambdaa = params[9]
    mu = params[10]
    y01 = vaccinated_initial_condition
    t1 = [0, 14, 26, 28] 
    yv1 = odeint(model, y01, t1, args=(beta, Delta, alpha, p, K, v, gamma, rho, N, lambdaa, mu), mxstep=2000)
    y02 = yv1[-1,:]
    #print(yv1)
    #print(y02)
    y02[3] = 10**(4.5)
    y02[4] = 10**(4.5)
    t2 = [28, 30, 30.5, 31, 31.5, 32, 32.5, 33, 33.5, 34, 34.5, 35, 35.5, 36, 36.5, 37, 37.5, 38, 38.5, 
                        39, 39.5, 40, 40.5, 41, 41.5, 56, 183] #make sure timesteps are right
    #print(t2)
    yv2 = odeint(model, y02, t2, args=(beta, Delta, alpha, p, K, v, gamma, rho, N, lambdaa, mu), mxstep=2000)
    #yv2_log = np.log(yv2)
   # print(len(yv2))
    #print(yv2[-2:-1, 6])
    #print(len(yv2[1:22, 3]))
    #print(len(Vinf_data))
    SSRinf = np.sum((yv2[1:22, 3] - Vinf_data)**2)
    #SSRinf_nonan = SSRinf[np.logical_not(np.isnan(SSRinf))]
    #print(SSRinf_nonan)
    #print(len(yv2[1:22, 4]))
    SSRrna = np.sum((yv2[1:22, 4] - Vrna_data)**2)
    #print(SSRrna)
    #print(np.sum((yv2[-2:-1, 6] - Ab_data[-2:-1])**2))
    SSRab = np.sum((yv1[1:3, 6] - Ab_data[1:3])**2) + np.sum((yv2[-2:-1, 6] - Ab_data[-2:-1])**2)
    + np.sum((yv2[7,6] - Ab_data[3])**2) + np.sum((yv2[17,6] - Ab_data[4])**2)
    #print(SSRab)
    return (SSRinf + SSRrna + SSRab)

- could split up yv1 and yv2
    - then do you run ssr's through yv1 then separately through yv2 and after yv1, you have new initial guesses for yv2? 
- or keep yv1 and yv2 together but run each ssr separately through it 
    - then would you add the ssr's together when running through the second model with the best-guess parameters? 
- lit search for parameters for antibodies for RSV 

In [ ]:
def ssr_vacc_inf(params): #list parameters
    beta = params[0]
    Delta = params[1]
    alpha = params[2]
    p = params[3]
   # cinf = params[4]
    K = params[4]
    v = params[5]
    #crna = params[7]
    gamma = params[6]
    rho = params[7]
    N = params[8]
    lambdaa = params[9]
    mu = params[10]
    y01 = vaccinated_initial_condition
    t1 = [0, 14, 26, 28] 
    yv1 = odeint(model, y01, t1, args=(beta, Delta, alpha, p, K, v, gamma, rho, N, lambdaa, mu), mxstep=2000)
    y02 = yv1[-1,:]
    y02[3] = 10**(4.5)
    y02[4] = 10**(4.5)
    t2 = [28, 30, 30.5, 31, 31.5, 32, 32.5, 33, 33.5, 34, 34.5, 35, 35.5, 36, 36.5, 37, 37.5, 38, 38.5, 
                        39, 39.5, 40, 40.5, 41, 41.5, 56, 183] #make sure timesteps are right
    yv2 = odeint(model, y02, t2, args=(beta, Delta, alpha, p, K, v, gamma, rho, N, lambdaa, mu), mxstep=2000)
    SSRinf = np.sum((yv2[1:22, 3] - Vinf_data)**2)
    return (SSRinf)

result_vacc_inf = minimize(ssr_vacc_inf, x0=initial_guess, method='Nelder-Mead') #nelder-mead
#print(result_vacc_inf.x)

 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.5454771487530D+02   r2 =  0.3412685869184D-14
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.5454771487530D+02   r2 =  0.3412685869184D-14
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.5454771487530D+02   r2 =  0.3412685869184D-14
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.5454771487530D+02   r2 =  0.3412685869184D-14
 lsoda--  warning..internal t (=r1) 

In [ ]:
def ssr_vacc_rna(params): #list parameters
    beta = params[0]
    Delta = params[1]
    alpha = params[2]
    p = params[3]
    #cinf = params[4]
    K = params[4]
    v = params[5]
    #crna = params[7]
    gamma = params[6]
    rho = params[7]
    N = params[8]
    lambdaa = params[9]
    mu = params[10]
    y01 = vaccinated_initial_condition
    t1 = [0, 14, 26, 28] 
    yv1 = odeint(model, y01, t1, args=(beta, Delta, alpha, p, K, v, gamma, rho, N, lambdaa, mu), mxstep=2000)
    y02 = yv1[-1,:]
    y02[3] = 10**(4.5)
    y02[4] = 10**(4.5)
    t2 = [28, 30, 30.5, 31, 31.5, 32, 32.5, 33, 33.5, 34, 34.5, 35, 35.5, 36, 36.5, 37, 37.5, 38, 38.5, 
                        39, 39.5, 40, 40.5, 41, 41.5, 56, 183] #make sure timesteps are right
    yv2 = odeint(model, y02, t2, args=(beta, Delta, alpha, p, K, v, gamma, rho, N, lambdaa, mu), mxstep=2000)
    SSRrna = np.sum((yv2[1:22, 4] - Vrna_data)**2)
    return (SSRrna)

result_vacc_rna = minimize(ssr_vacc_rna, x0=initial_guess, method='Nelder-Mead') #nelder-mead
print(result_vacc_rna.x)

In [ ]:
def ssr_vacc_ab(params): #list parameters
    beta = params[0]
    Delta = params[1]
    alpha = params[2]
    p = params[3]
    #cinf = params[4]
    K = params[4]
    v = params[5]
    #crna = params[7]
    gamma = params[6]
    rho = params[7]
    N = params[8]
    lambdaa = params[9]
    mu = params[10]
    y01 = vaccinated_initial_condition
    t1 = [0, 14, 26, 28] 
    yv1 = odeint(model, y01, t1, args=(beta, Delta, alpha, p, K, v, gamma, rho, N, lambdaa, mu), mxstep=2000)
    y02 = yv1[-1,:]
    y02[3] = 10**(4.5)
    y02[4] = 10**(4.5)
    t2 = [28, 30, 30.5, 31, 31.5, 32, 32.5, 33, 33.5, 34, 34.5, 35, 35.5, 36, 36.5, 37, 37.5, 38, 38.5, 
                        39, 39.5, 40, 40.5, 41, 41.5, 56, 183] #make sure timesteps are right
    yv2 = odeint(model, y02, t2, args=(beta, Delta, alpha, p, K, v, gamma, rho, N, lambdaa, mu), mxstep=2000)
    SSRab = np.sum((yv1[1:3, 6] - Ab_data[1:3])**2) + np.sum((yv2[-2:-1, 6] - Ab_data[-2:-1])**2)
    + np.sum((yv2[7,6] - Ab_data[3])**2) + np.sum((yv2[17,6] - Ab_data[4])**2)
    return (SSRab)

result_vacc_ab = minimize(ssr_vacc_ab, x0=initial_guess, method='Nelder-Mead') #nelder-mead
print(result_vacc_ab.x)

In [ ]:
result_vacc = minimize(ssr_vacc, x0=initial_guess, method='Nelder-Mead') #nelder-mead
print(result_vacc.x)

In [ ]:
def model2(variables, t):
    T = variables[0]
    I = variables[1]
    Va = variables[2]
    Vinf = variables[3]
    Vrna = variables[4]
    M = variables[5]
    A = variables[6]
    dTdt = -result_vacc_ab.x[0] * T * Vinf #actual variables 
    dIdt = result_vacc_ab.x[0] * T * Vinf - result_vacc_ab.x[1] * I #make sure these are right
    dVadt = -result_vacc_ab.x[2] * Va
    dVinfdt = result_vacc_ab.x[3] * I - cinf * Vinf - result_vacc_ab.x[4] * A * Vinf
    dVrnadt = result_vacc_ab.x[5] * result_vacc_ab.x[3] * I - crna * Vrna - result_vacc_ab.x[5] * A * Vrna
    dMdt = (result_vacc_ab.x[6] * (Vrna + Va) + result_vacc_ab.x[7] * M) * (1 - M / result_vacc_ab.x[8])
    dAdt = result_vacc_ab.x[9] * (Vrna + Va) * M * (1 - A / result_vacc_ab.x[8]) - result_vacc_ab.x[10] * A
    return [dTdt, dIdt, dVadt, dVinfdt, dVrnadt, dMdt, dAdt]

sol1 = odeint(model2, vaccinated_initial_condition, t1)
sol2 = odeint(model2, vaccinated_initial_condition, t2)

def model3(variables, t):
    T = variables[0]
    I = variables[1]
    Va = variables[2]
    Vinf = variables[3]
    Vrna = variables[4]
    M = variables[5]
    A = variables[6]
    dTdt = -result_vacc_ab.x[0] * T * Vinf #actual variables 
    dIdt = result_vacc_ab.x[0] * T * Vinf - result_vacc_ab.x[1] * I #make sure these are right
    dVadt = -result_vacc_ab.x[2] * Va
    dVinfdt = result_vacc_ab.x[3] * I - cinf * Vinf - result_vacc_ab.x[4] * A * Vinf
    dVrnadt = result_vacc_ab.x[5] * result_vacc_ab.x[3] * I - crna * Vrna - result_vacc_ab.x[5] * A * Vrna
    dMdt = (result_vacc_ab.x[6] * (Vrna + Va) + result_vacc_ab.x[7] * M) * (1 - M / result_vacc_ab.x[8])
    dAdt = result_vacc_ab.x[9] * (Vrna + Va) * M * (1 - A / result_vacc_ab.x[8]) - result_vacc_ab.x[10] * A
    return [dTdt, dIdt, dVadt, dVinfdt, dVrnadt, dMdt, dAdt]

sol_ab = odeint(model2, vaccinated_initial_condition, np.linspace(0, 180, num=5))
sol_rna = odeint(model3, vaccinated_initial_condition, np.linspace(0, 180, num=21))
plt.plot(np.linspace(0, 180, num=5), sol_ab)
plt.plot(np.linspace(0, 180, num=21), sol_rna)

plt.plot(t1, sol1)
plt.plot(t2, sol2)
plt.plot(np.linspace(0, 180, num=5), Ab_data, 'o')
plt.plot(np.linspace(0, 180, num=21), Vrna_data, 'o')
plt.yscale('log')
plt.show()

In [ ]:
plt.plot(np.linspace(0, 180, num=21), Vinf_data, 'o', label='v_infected')
plt.plot(np.linspace(0, 180, num=21), Vrna_data, 'o', label='v_rna')
plt.plot(np.linspace(0, 180, num=5), Ab_data, 'o', label='antibodies')
plt.plot(t1, sol1)
plt.plot(t2, sol2)
plt.yscale('log')
plt.legend()
plt.show()

- dVa/dt = -alpha * Va
- dM/dt = (gamma * Va + rho * M)(1 - M/N), will be a logistic curveish 
- skip temp antibodies 
- assume everything is a memory antibody 
- gets rid of two params 